In [11]:
import pandas as pd
import sqlite3 as s3
from sklearn.cluster import KMeans
import math
import matplotlib.pyplot as plt
import warnings
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
warnings.simplefilter('ignore')

In [12]:
go_sales = s3.connect('go_sales.sqlite')
go_crm = s3.connect('go_crm.sqlite')
df = pd.read_sql_query('SELECT * FROM product', go_sales)
df = pd.merge(df, pd.read_sql_query('SELECT * FROM order_details', go_sales), on='PRODUCT_NUMBER')
df

,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION,TRIAL888,ORDER_DETAIL_CODE,ORDER_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE,TRIAL879
0,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100085,8474,40,4.01,6.03,6.03,T
1,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100086,8470,40,4.01,6.03,6.03,T
2,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100087,8464,48,4.01,6.03,6.03,T
3,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100088,9254,66,4.01,6.03,6.03,T
4,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100089,8472,88,4.01,6.03,6.03,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,98478,8210,30,1.97,4.94,4.94,T
43059,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99566,8246,20,2,5,3.95,T
43060,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99567,8239,18,2,5,5,T
43061,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99568,8270,26,2,5,5,T


In [13]:
df = df.loc[:, ['PRODUCT_NUMBER', 'PRODUCT_NAME', 'ORDER_NUMBER', 'QUANTITY', 'UNIT_SALE_PRICE']]
df

,PRODUCT_NUMBER,PRODUCT_NAME,ORDER_NUMBER,QUANTITY,UNIT_SALE_PRICE
0,1,TrailChef Water Bag,8474,40,6.03
1,1,TrailChef Water Bag,8470,40,6.03
2,1,TrailChef Water Bag,8464,48,6.03
3,1,TrailChef Water Bag,9254,66,6.03
4,1,TrailChef Water Bag,8472,88,6.03
...,...,...,...,...,...
43058,99,Aloe Relief,8210,30,4.94
43059,99,Aloe Relief,8246,20,3.95
43060,99,Aloe Relief,8239,18,5
43061,99,Aloe Relief,8270,26,5


In [14]:
df = pd.get_dummies(df, columns=['PRODUCT_NAME'])
df

,PRODUCT_NUMBER,ORDER_NUMBER,QUANTITY,UNIT_SALE_PRICE,PRODUCT_NAME_Aloe Relief,PRODUCT_NAME_Bear Edge,PRODUCT_NAME_Bear Survival Edge,PRODUCT_NAME_Blue Steel Max Putter,PRODUCT_NAME_Blue Steel Putter,PRODUCT_NAME_BugShield Extreme,...,PRODUCT_NAME_TrailChef Canteen,PRODUCT_NAME_TrailChef Cook Set,PRODUCT_NAME_TrailChef Cup,PRODUCT_NAME_TrailChef Deluxe Cook Set,PRODUCT_NAME_TrailChef Double Flame,PRODUCT_NAME_TrailChef Kettle,PRODUCT_NAME_TrailChef Kitchen Kit,PRODUCT_NAME_TrailChef Single Flame,PRODUCT_NAME_TrailChef Utensils,PRODUCT_NAME_TrailChef Water Bag
0,1,8474,40,6.03,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1,8470,40,6.03,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,1,8464,48,6.03,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,1,9254,66,6.03,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,1,8472,88,6.03,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,99,8210,30,4.94,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
43059,99,8246,20,3.95,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
43060,99,8239,18,5,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
43061,99,8270,26,5,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [15]:
te = TransactionEncoder()
te_ary = te.fit(transaction_list).transform(transaction_list)
one_hot_df = pd.DataFrame(te_ary, columns=te.columns_)

NameError: name 'transaction_list' is not defined